In [1]:
import numpy as np
import pandas as pd

import os
import sys
sys.path.insert(0, '..')
import ccal

%matplotlib inline
# %config InlineBackend.figure_formats = {'svg',}

==================== Computational Cancer Analysis Library ====================
<12:53:33> Checking library dependencies ...
<12:53:33> Using the following libraries (in addition to the Anaconda libraries):
<12:53:33> 	scipy (v0.18.0)
<12:53:33> 	statsmodels (v0.6.1)
<12:53:33> 	scikit-learn (v0.17.1)
<12:53:33> 	seaborn (v0.7.1)
<12:53:33> 	rpy2 (v2.8.2)
<12:53:33> 	pandas (v0.18.1)
<12:53:33> 	matplotlib (v1.5.1)
<12:53:33> 	numpy (v1.11.1)
<12:53:33> Planted a random seed 20121020.


# Get states

In [2]:
h = ccal.support.read_gct('/home/cyborg/hematopoietic_cancer/result/ccle_hema_r/k9/ccle_hema_k_9_H.gct')
ks = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 25, 30]
n_clusterings = 10
filepath_prefix = 'output/ccle_hema_k_9_H_{}clustering'.format(n_clusterings)

labels, memberships, cophenetic_correlation = ccal.analyze.define_states(h, ks, n_clusterings=n_clusterings,
                                                                             filepath_prefix=filepath_prefix)

# Map Onco-GPS

In [ ]:
labels = pd.read_csv(filepath_prefix + '_labels.txt', index_col=0, sep='\t')
memberships = ccal.support.read_gct(filepath_prefix + '_memberships.gct')
annotations = ccal.support.make_random_features(1, memberships.shape[1])

In [ ]:
for n_components in range(4, min(h.shape[0], 21), 3):
    #h = ccal.support.make_random_features(n_components, 100)
    h = h.iloc[:n_components, :]
    for state in range(1, 21, 5):
        for annotation_type in ['continuous', 'categorical', 'binary']:
            ccal.onco_gps.make_map(h,
                                   np.random.choice(list(range(state)), h.shape[1]),
                                   annotations=np.random.choice(range(2), h.shape[1]),
                                   annotation_type=annotation_type)#,
                                   #background_markersize=0, background_mask_markersize=0)

<12:53:33> Creating Onco-GPS with 166 samples, 4 components, and 1 states [0] ...
<12:53:33> '-0-' normalizing pandas object with axis=1 ...
<12:53:33> '0-1' normalizing pandas object with axis=1 ...
<12:53:33> Converting association to distance (1 - association) ...
<12:53:34> '0-1' normalizing pandas object with axis=0 ...
<12:53:34> Computing the sample_stretch_factor ...
<12:53:34> 	Modeled H columns by 0.017482260545828084e^(1.1271366603000295x) + 0.0005279448319646031.
<12:53:34> 	sample_stretch_factor = 2.127.
<12:53:46> '-0-' normalizing pandas object with axis=all ...
